# Distributed Training with Keras

**Learning Objectives**

1. How to define distribution strategy and set input pipeline.
2. How to create the Keras model.
3. How to define the callbacks.
4. How to train and evaluate the model.

## Introduction

The `tf.distribute.Strategy` API provides an abstraction for distributing your training
across multiple processing units. The goal is to allow users to enable distributed training using existing models and training code, with minimal changes.

This notebook uses the `tf.distribute.MirroredStrategy`, which
does in-graph replication with synchronous training on many GPUs on one machine.
Essentially, it copies all of the model's variables to each processor.
Then, it uses [all-reduce](http://mpitutorial.com/tutorials/mpi-reduce-and-allreduce/) to combine the gradients from all processors and applies the combined value to all copies of the model.

`MirroredStrategy` is one of several distribution strategy available in TensorFlow core. You can read about more strategies at [distribution strategy guide](https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/distributed_training.ipynb).

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/keras.ipynb).

### Keras API

This example uses the `tf.keras` API to build the model and training loop. For custom training loops, see the [tf.distribute.Strategy with training loops](training_loops.ipynb) tutorial.

## Import dependencies

In [1]:
# Import TensorFlow and TensorFlow Datasets

import tensorflow_datasets as tfds
import tensorflow as tf

import os

In [2]:
# Here we'll show the currently installed version of TensorFlow
print(tf.__version__)

2.6.5


## Download the dataset

Download the MNIST dataset and load it from [TensorFlow Datasets](https://www.tensorflow.org/datasets). This returns a dataset in `tf.data` format.

Setting `with_info` to `True` includes the metadata for the entire dataset, which is being saved here to `info`.
Among other things, this metadata object includes the number of train and test examples. 


In [8]:
import tensorflow as tf

# Load dataset from CSV using tf.data.experimental.make_csv_dataset
dataset = tf.data.experimental.make_csv_dataset(
    'data/score_test.csv',  # Path to the CSV file
    batch_size=32,          # Batch size for the dataset
    label_name='Graduated',  # Specify the label column if applicable
    num_epochs=1,           # Number of epochs to go through the dataset
    shuffle=True            # Shuffle the dataset
)

# Example of how to iterate through the dataset
for batch in dataset.take(1):
    features, label = batch
    print(features)
    print(label)

# If you're working with the MNIST dataset (separately)
import tensorflow_datasets as tfds
mnist_train, mnist_test = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)


2024-09-20 09:33:30.438474: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


OrderedDict([('Profession', <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'', b'Healthcare', b'Artist', b'Doctor', b'Engineer', b'Artist',
       b'Doctor', b'Healthcare', b'Artist', b'Entertainment', b'Lawyer',
       b'Marketing', b'Healthcare', b'Doctor', b'Entertainment',
       b'Artist', b'Artist', b'Artist', b'Artist', b'Artist', b'Engineer',
       b'Artist', b'Engineer', b'Doctor', b'Healthcare', b'Entertainment',
       b'Artist', b'Artist', b'Artist', b'Artist', b'Artist',
       b'Executive'], dtype=object)>), ('Work_Experience', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0., 1., 1., 1., 0., 0., 4., 0., 0., 1., 1., 0., 0., 0., 0., 8., 0.,
       4., 2., 0., 4., 0., 0., 6., 1., 0., 0., 0., 0., 0., 0., 1.],
      dtype=float32)>), ('Family_Size', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([3., 4., 2., 3., 5., 4., 3., 4., 2., 4., 2., 4., 5., 3., 4., 1., 3.,
       3., 9., 2., 2., 1., 0., 0., 2., 2., 2., 3., 3., 6., 4., 2.],
      dtype=float32)>), 

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /home/jupyter/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


## Define distribution strategy

Create a `MirroredStrategy` object. This will handle distribution, and provides a context manager (`tf.distribute.MirroredStrategy.scope`) to build your model inside.

In [9]:
# Synchronous training across multiple replicas on one machine.
# TODO: Your code goes here
import tensorflow as tf

# Create a MirroredStrategy object to distribute the training across GPUs
strategy = tf.distribute.MirroredStrategy()

# Use the strategy's scope to build and compile your model
with strategy.scope():
    # Define your model here
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Compile the model inside the strategy's scope
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Now the model is distributed across multiple GPUs, and synchronous training will be applied


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [10]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


## Setup input pipeline

When training a model with multiple GPUs, you can use the extra computing power effectively by increasing the batch size. In general, use the largest batch size that fits the GPU memory, and tune the learning rate accordingly.

In [12]:
import tensorflow_datasets as tfds
import tensorflow as tf

# Assuming you're working with MNIST, load the dataset along with its info
dataset_name = 'mnist'
(train_dataset, test_dataset), info = tfds.load(
    dataset_name,
    split=['train', 'test'],  # Load the training and test sets
    as_supervised=True,       # Returns tuple (input, label)
    with_info=True            # Also return the dataset metadata
)

# Get the number of training and test examples from the dataset info
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

# Define buffer size and batch size for training
BUFFER_SIZE = 10000

# Define batch size per replica (for distributed strategy)
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

# Prepare datasets for training
train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# Example of distributed strategy (assuming you already defined `strategy`)
with strategy.scope():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=10)


2024-09-20 09:36:08.107915: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:461] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.
2024-09-20 09:36:08.165305: W tensorflow/core/framework/dataset.cc:679] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


Epoch 1/10


Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup ca

 25/938 [..............................] - ETA: 6s - loss: 36.5247 - accuracy: 0.5694  

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup ca

...

830/938 [=========================>....] - ETA: 0s - loss: 3.8224 - accuracy: 0.8710

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


938/938 [==============================] - 10s 7ms/step - loss: 3.4645 - accuracy: 0.8719
Epoch 2/10
938/938 [==============================] - 3s 3ms/step - loss: 0.4620 - accuracy: 0.9062
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2912 - accuracy: 0.9284
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2393 - accuracy: 0.9408
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2020 - accuracy: 0.9471
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.1821 - accuracy: 0.9513
Epoch 7/10
938/938 [==============================] - 3s 3ms/step - loss: 0.1722 - accuracy: 0.9550
Epoch 8/10
938/938 [==============================] - 3s 3ms/step - loss: 0.1639 - accuracy: 0.9564
Epoch 9/10
 49/938 [>.............................] - ETA: 2s - loss: 0.1420 - accuracy: 0.9643

2024-09-20 09:36:39.249203: W tensorflow/core/framework/dataset.cc:679] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


938/938 [==============================] - 3s 3ms/step - loss: 0.1578 - accuracy: 0.9592
Epoch 10/10
938/938 [==============================] - 3s 3ms/step - loss: 0.1430 - accuracy: 0.9622


Pixel values, which are 0-255, [have to be normalized to the 0-1 range](https://en.wikipedia.org/wiki/Feature_scaling). Define this scale in a function.

In [13]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

Apply this function to the training and test data, shuffle the training data, and [batch it for training](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch). Notice we are also keeping an in-memory cache of the training data to improve performance.


In [14]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

## Create the model

Create and compile the Keras model in the context of `strategy.scope`.

In [16]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)  # Final layer for classification
    ])

    # Compile the model
    model.compile(
        optimizer='adam',  # Adam optimizer
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # Loss function for classification
        metrics=['accuracy']  # Track accuracy during training
    )


## Define the callbacks


The callbacks used here are:

*   *TensorBoard*: This callback writes a log for TensorBoard which allows you to visualize the graphs.
*   *Model Checkpoint*: This callback saves the model after every epoch.
*   *Learning Rate Scheduler*: Using this callback, you can schedule the learning rate to change after every epoch/batch.

For illustrative purposes, add a print callback to display the *learning rate* in the notebook.

In [ ]:
# Define the checkpoint directory to store the checkpoints

# TODO: Your code goes here
# Name of the checkpoint files
# TODO: Your code goes here
import tensorflow as tf
import os

# Define log directory for TensorBoard
log_dir = os.path.join("logs", "fit")

# Define a custom callback to print the learning rate during training
class PrintLearningRate(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        print(f"\nLearning rate at epoch {epoch+1} is {lr.numpy()}")

# Define the learning rate scheduler callback
def scheduler(epoch, lr):
    # Reduce learning rate by half every 10 epochs
    if epoch % 10 == 0 and epoch != 0:
        return lr * 0.5
    else:
        return lr

# Define the list of callbacks
callbacks = [
    # TensorBoard callback
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
    
    # ModelCheckpoint callback to save the model at each epoch
    tf.keras.callbacks.ModelCheckpoint(
        filepath="model_epoch_{epoch}.h5",  # Filepath to save the model
        save_best_only=True,  # Save only if it's the best model so far
        monitor='val_loss',  # Monitor validation loss
        verbose=1
    ),
    
    # LearningRateScheduler callback
    tf.keras.callbacks.LearningRateScheduler(scheduler),
    
    # Print learning rate callback
    PrintLearningRate()
]

# Build the model inside the distribution strategy's scope
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Initial learning rate
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

# Train the model and apply the callbacks
model.fit(train_dataset, epochs=50, validation_data=test_dataset, callbacks=callbacks)


2024-09-20 09:47:13.617275: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2024-09-20 09:47:13.617336: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2024-09-20 09:47:13.620435: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2024-09-20 09:47:13.689170: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:461] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.
2024-09-20 09:47:13.751252: W tensorflow/core/framework/dataset.cc:679] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


Epoch 1/50


Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup ca

  1/938 [..............................] - ETA: 56:05 - loss: 2.2867 - accuracy: 0.1094

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup ca

...

261/938 [=======>......................] - ETA: 17s - loss: 0.3975 - accuracy: 0.8867

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup ca

268/938 [=======>......................] - ETA: 17s - loss: 0.3918 - accuracy: 0.8886

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup ca

272/938 [=======>......................] - ETA: 17s - loss: 0.3888 - accuracy: 0.8894

In [12]:
# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [13]:
# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

In [14]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

## Train and evaluate

Now, train the model in the usual way, calling `fit` on the model and passing in the dataset created at the beginning of the tutorial. This step is the same whether you are distributing the training or not.


In [15]:
# Train the model with the new callback
# TODO: Your code goes here


Epoch 1/12
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


  3/938 [..............................] - ETA: 41s - loss: 2.2821 - accuracy: 0.1172    WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0062s vs `on_train_batch_end` time: 0.0128s). Check your callbacks.


938/938 [==============================] - 16s 7ms/step - loss: 0.4077 - accuracy: 0.8858

Learning rate for epoch 1 is 0.0010000000474974513
Epoch 2/12
938/938 [==============================] - 3s 4ms/step - loss: 0.0711 - accuracy: 0.9787

Learning rate for epoch 2 is 0.0010000000474974513
Epoch 3/12
938/938 [==============================] - 3s 4ms/step - loss: 0.0459 - accuracy: 0.9861

Learning rate for epoch 3 is 0.0010000000474974513
Epoch 4/12
938/938 [==============================] - 3s 4ms/step - loss: 0.0279 - accuracy: 0.9921

Learning rate for epoch 4 is 9.999999747378752e-05
Epoch 5/12
938/938 [==============================] - 3s 4ms/step - loss: 0.0237 - accuracy: 0.9935

Learning rate for epoch 5 is 9.999999747378752e-05
Epoch 6/12
938/938 [==============================] - 3s 4ms/step - loss: 0.0211 - accuracy: 0.9943

Learning rate for epoch 6 is 9.999999747378752e-05
Epoch 7/12
938/938 [==============================] - 3s 4ms/step - loss: 0.0204 - accuracy: 0.994

As you can see below, the checkpoints are getting saved.

In [16]:
# check the checkpoint directory
!ls {checkpoint_dir}

checkpoint		     ckpt_4.data-00000-of-00001
ckpt_10.data-00000-of-00001  ckpt_4.index
ckpt_10.index		     ckpt_5.data-00000-of-00001
ckpt_11.data-00000-of-00001  ckpt_5.index
ckpt_11.index		     ckpt_6.data-00000-of-00001
ckpt_12.data-00000-of-00001  ckpt_6.index
ckpt_12.index		     ckpt_7.data-00000-of-00001
ckpt_1.data-00000-of-00001   ckpt_7.index
ckpt_1.index		     ckpt_8.data-00000-of-00001
ckpt_2.data-00000-of-00001   ckpt_8.index
ckpt_2.index		     ckpt_9.data-00000-of-00001
ckpt_3.data-00000-of-00001   ckpt_9.index
ckpt_3.index


To see how the model perform, load the latest checkpoint and call `evaluate` on the test data.

Call `evaluate` as before using appropriate datasets.

In [17]:
# Loads the weights
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 3s 8ms/step - loss: 0.0381 - accuracy: 0.9874
Eval loss: 0.03809080645442009, Eval Accuracy: 0.9873999953269958


To see the output, you can download and view the TensorBoard logs at the terminal.

```
$ tensorboard --logdir=path/to/log-directory
```

In [18]:
!ls -sh ./logs

total 4.0K
4.0K train


## Export to SavedModel

Export the graph and the variables to the platform-agnostic SavedModel format. After your model is saved, you can load it with or without the scope.


In [19]:
path = 'saved_model/'

In [20]:
# Save the entire model as a SavedModel.
# TODO: Your code goes here


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


Load the model without `strategy.scope`.

In [21]:
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 1s 3ms/step - loss: 0.0361 - accuracy: 0.9892
Eval loss: 0.03809080645442009, Eval Accuracy: 0.9873999953269958


Load the model with `strategy.scope`.

In [22]:
# Recreate the exact same model, including its weights and the optimizer
with strategy.scope():
  replicated_model = tf.keras.models.load_model(path)
  replicated_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])

  eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
  print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 3s 4ms/step - loss: 0.0361 - accuracy: 0.9892
Eval loss: 0.03809080645442009, Eval Accuracy: 0.9873999953269958


### Examples and Tutorials
Here are some examples for using distribution strategy with keras fit/compile:
1. [Transformer](https://github.com/tensorflow/models/blob/master/official/nlp/transformer/transformer_main.py) example trained using `tf.distribute.MirroredStrategy`
2. [NCF](https://github.com/tensorflow/models/blob/master/official/recommendation/ncf_keras_main.py) example trained using `tf.distribute.MirroredStrategy`.

More examples listed in the [Distribution strategy guide](https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/distributed_training.ipynb#examples_and_tutorials)

## Next steps

* Read the [distribution strategy guide](https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/distributed_training.ipynb).
* Read the [Distributed Training with Custom Training Loops](https://raw.githubusercontent.com/tensorflow/docs/master/site/en/tutorials/distribute/custom_training.ipynb) tutorial.
* Visit the [Performance section](https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/function.ipynb) in the guide to learn more about other strategies and [tools](https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/profiler.md) you can use to optimize the performance of your TensorFlow models.

Note: `tf.distribute.Strategy` is actively under development and we will be adding more examples and tutorials in the near future. Please give it a try. We welcome your feedback via [issues on GitHub](https://github.com/tensorflow/tensorflow/issues/new).